<a href="https://colab.research.google.com/github/adampotton/Group-3-gotta-catch-em-all-/blob/main/Model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
from torchvision.transforms import ToPILImage
import matplotlib.pyplot as plt
import numpy as np
from google.colab import files
from PIL import Image
import random
import os
from google.colab import drive
from sklearn.model_selection import train_test_split
drive.mount('/content/drive')
torch.manual_seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Mounted at /content/drive


In [3]:
root_directory = "/content/drive/My Drive/5Pokemon"

# Create a dictionary to store image arrays, using folder names as keys
image_arrays_dict = {}

# Define a common size for resizing images
common_size = (224, 224, 3)  # Common size for RGB images

# Loop through each folder in the root directory
for folder_name in os.listdir(root_directory):
    folder_path = os.path.join(root_directory, folder_name)
    if os.path.isdir(folder_path):
        image_arrays = []
        for filename in os.listdir(folder_path):
            if filename.lower().endswith((".jpg", ".png")):
                image_path = os.path.join(folder_path, filename)
                img = Image.open(image_path).convert("RGB")  # Convert to RGB format
                img_resized = img.resize((common_size[1], common_size[0]), Image.ANTIALIAS)
                img_array = np.array(img_resized)
                image_arrays.append(img_array)
        image_arrays = np.array(image_arrays)
        image_arrays_dict[folder_name] = image_arrays

# Combine all image arrays into a single array
all_images = np.concatenate(list(image_arrays_dict.values()), axis=0)


for folder_name, image_array in image_arrays_dict.items():
    print(f"Shape of the image array for folder '{folder_name}': {image_array.shape}")

<ipython-input-3-896b65ba6408>:18: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img_resized = img.resize((common_size[1], common_size[0]), Image.ANTIALIAS)


Shape of the image array for folder 'Charmander': (51, 224, 224, 3)
Shape of the image array for folder 'Pikachu': (60, 224, 224, 3)
Shape of the image array for folder 'Squirtle': (52, 224, 224, 3)
Shape of the image array for folder 'Eevee': (37, 224, 224, 3)
Shape of the image array for folder 'Bulbasaur': (41, 224, 224, 3)


In [4]:
pokemon_classes = ['Charmander', 'Squirtle', 'Pikachu', 'Eevee', 'Bulbasaur']
num_images_per_class = [51,52 , 60, 37, 41]
all_labels = np.concatenate([np.full(num, i) for i, num in enumerate(num_images_per_class)])

train_images, test_images, train_labels, test_labels = train_test_split(all_images, all_labels, test_size=0.2, random_state=42, stratify=all_labels)
# Assuming you have the image arrays stored in 'all_images' and corresponding labels in 'all_labels'
# Make sure 'all_images' and 'all_labels' are NumPy arrays
print(train_images.shape)
train_images_tensor = torch.tensor(train_images, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)
test_images_tensor = torch.tensor(test_images, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.long)

# Create PyTorch datasets for training and testing
train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_images_tensor, test_labels_tensor)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
# Create data loaders

# Number of classes for your Pokémon images
num_classes = 5

# Number of input channels in your Pokémon images
num_input_channels = 3  # Assuming RGB images


# Define a simpler model
model = nn.Sequential(
    nn.Conv2d(3, 16, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.MaxPool2d(2),
    nn.Conv2d(16, 32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(32 * 56 * 56, 128),
    nn.ReLU(),
    nn.Linear(128, num_classes)
)

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Train the model
num_epochs = 15

for epoch in range(num_epochs):
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images.permute(0, 3, 1, 2))

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the accuracy after each epoch
    with torch.no_grad():
        total = 0
        correct = 0
        for images, labels in train_loader:
            outputs = model(images.permute(0, 3, 1, 2))
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)

            # Adjust the size of true labels tensor dynamically
            correct += (predicted[:len(labels)] == labels).sum().item()

        accuracy = correct / total * 100
        print(f'Epoch [{epoch+1}/{num_epochs}], Accuracy: {accuracy:.2f}%')


(192, 224, 224, 3)
Epoch [1/15], Accuracy: 25.52%
Epoch [2/15], Accuracy: 22.92%
Epoch [3/15], Accuracy: 47.40%
Epoch [4/15], Accuracy: 68.75%
Epoch [5/15], Accuracy: 92.71%
Epoch [6/15], Accuracy: 98.44%
Epoch [7/15], Accuracy: 98.96%
Epoch [8/15], Accuracy: 98.96%
Epoch [9/15], Accuracy: 98.96%
Epoch [10/15], Accuracy: 98.96%
Epoch [11/15], Accuracy: 98.96%
Epoch [12/15], Accuracy: 98.96%


KeyboardInterrupt: 

In [1]:
#test_images_tensor = torch.tensor(all_images, dtype=torch.float32)
#test_labels_tensor = torch.tensor(all_labels, dtype=torch.long)

# Create a PyTorch dataset for testing
#test_dataset = TensorDataset(test_images_tensor, test_labels_tensor)

#test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

def display_random_pokemon_prediction():
    with torch.no_grad():
        # Choose a random index from the Pokémon test dataset
        index_to_display = random.randint(0, len(test_dataset) - 1)

        # Extract the image and label from the Pokémon test dataset
        image, label = test_dataset[index_to_display]
        image = image.unsqueeze(0).to(device)  # Add batch dimension and move to device

        # Ensure that the image has 3 or fewer channels
        if image.shape[-1] > 3:
            # If more than 3 channels, consider only the first 3 channels (assuming RGB)
            image = image[:, :, :]

        # Convert the PyTorch tensor to a NumPy array with the correct data type
        np_image = (image.squeeze().cpu().numpy() * 255).astype(np.uint8)
        np_image = 255 - np_image
        # Convert the resized image tensor to a PIL image for display
        to_pil = ToPILImage()
        pil_image = to_pil(np_image)

        # Resize image to standard size (adjust as needed)
        resized_image = transforms.Resize((224, 224))(pil_image)

        # Convert the resized PIL image back to a PyTorch tensor
        resized_image_tensor = transforms.ToTensor()(resized_image).unsqueeze(0).to(device)

        # Forward pass
        logits = model(resized_image_tensor).squeeze((-1, -2))
        predicted_label = torch.argmax(logits).item()

        # Display the image
        plt.imshow(resized_image)
        plt.title(f"Actual: {pokemon_classes[label]} (Class {label}), Predicted: {pokemon_classes[predicted_label]} (Class {predicted_label})")
        plt.show()

# Display the image and its predicted result
display_random_pokemon_prediction()


NameError: name 'torch' is not defined